In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import BayesianGaussianMixture
from sklearn.metrics import accuracy_score

In [4]:
# Read the dataset and preprocess as described
full_train = pd.read_csv("datasets/train_clean_pca_transformed.csv")
full_test = pd.read_csv("datasets/test_clean_pca_transformed.csv")

target_col = "Survived"
X = full_train.drop(target_col, axis=1)
y = full_train[target_col]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# X_train, y_train = X, y

# Create the Bayesian Gausian Mixture Classifier

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Create the GridSearchCV object
grid_search = GridSearchCV(
    knn_model, param_grid, n_jobs=1, cv=StratifiedKFold, scoring="accuracy", verbose=4
)

# Fit the model
grid_search.fit(X_train, y_train)

# Get the best parameters and best model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate accuracy on the test set
accuracy = accuracy_score(y_test, y_pred)

print(f"Best Parameters: {best_params}")
# print(f"Best Score: {grid_search.best_score_:.4f}")
print(f"Accuracy on Test Set: {accuracy:.4f}")

Fitting 7 folds for each of 1 candidates, totalling 7 fits
[CV 1/7] END algorithm=ball_tree, leaf_size=5, metric=euclidean, n_jobs=-1, n_neighbors=4, p=1, weights=uniform;, score=0.814 total time=   3.2s
[CV 2/7] END algorithm=ball_tree, leaf_size=5, metric=euclidean, n_jobs=-1, n_neighbors=4, p=1, weights=uniform;, score=0.853 total time=   0.0s
[CV 3/7] END algorithm=ball_tree, leaf_size=5, metric=euclidean, n_jobs=-1, n_neighbors=4, p=1, weights=uniform;, score=0.902 total time=   0.0s
[CV 4/7] END algorithm=ball_tree, leaf_size=5, metric=euclidean, n_jobs=-1, n_neighbors=4, p=1, weights=uniform;, score=0.863 total time=   0.0s
[CV 5/7] END algorithm=ball_tree, leaf_size=5, metric=euclidean, n_jobs=-1, n_neighbors=4, p=1, weights=uniform;, score=0.792 total time=   0.0s
[CV 6/7] END algorithm=ball_tree, leaf_size=5, metric=euclidean, n_jobs=-1, n_neighbors=4, p=1, weights=uniform;, score=0.851 total time=   0.0s
[CV 7/7] END algorithm=ball_tree, leaf_size=5, metric=euclidean, n_jobs

In [5]:
import joblib

joblib.dump(grid_search, 'grid_search_model.joblib')

['grid_search_model.joblib']

In [6]:
loaded_grid_search = joblib.load('grid_search_model.joblib')

In [7]:
for k, v in loaded_grid_search.cv_results_.items():
    print(k, v, sep="\t")

mean_fit_time	[0.00371443]
std_fit_time	[0.00190504]
mean_score_time	[0.4952848]
std_score_time	[1.15932513]
param_algorithm	['ball_tree']
param_leaf_size	[5]
param_metric	['euclidean']
param_n_jobs	[-1]
param_n_neighbors	[4]
param_p	[1]
param_weights	['uniform']
params	[{'algorithm': 'ball_tree', 'leaf_size': 5, 'metric': 'euclidean', 'n_jobs': -1, 'n_neighbors': 4, 'p': 1, 'weights': 'uniform'}]
split0_test_score	[0.81372549]
split1_test_score	[0.85294118]
split2_test_score	[0.90196078]
split3_test_score	[0.8627451]
split4_test_score	[0.79207921]
split5_test_score	[0.85148515]
split6_test_score	[0.86138614]
mean_test_score	[0.84804615]
std_test_score	[0.03305147]
rank_test_score	[1]


In [8]:
real_test = pd.read_csv("datasets/test_clean_pca_transformed.csv")

In [9]:
pred = pd.DataFrame(best_model.predict(real_test.drop(columns=["PassengerId"]))).rename(columns={0: "Survived"})

In [10]:
pd.concat([real_test[["PassengerId"]], pred], axis=1).to_csv("KNN_v1.csv", index=None)